In [1]:
import requests as req
import pandas as pd
import json
from PIL import Image
from io import BytesIO
import random
from tqdm import tqdm
from oracle_test import OracleTools
import my_utils as mu
import time
import multiprocessing as mp

In [2]:
class WrongParticipantNumber(Exception):
    def __init__(self) -> None:
        super().__init__("participant_number는 1~10 사이의 값으로 입력해주세요.")

In [3]:
with open('../json/legendItem.json', 'rt', encoding='utf8') as f:
    legend_items = [int(a) for a in json.loads(f.read())]
with open('../json/mythicItem.json', 'rt', encoding='utf8') as f:
    mythic_itmes = [int(a) for a in json.loads(f.read())]
with open('D:\\7_LOL Project\\dragontail-13.8.1\\13.8.1\\data\\ko_KR\\item.json', 'rt', encoding='utf8') as f:
    items = json.loads(f.read())

In [30]:
def rawdataLoader(datas: str):
    return json.loads(datas.replace("'", '"'))

In [3]:
def itemId2Name(item_id: int):
    return items['data'][str(item_id)]['name']

In [ ]:
sample = mu.getSampleData('gold', 1, 1, '')
mat = sample.iloc[0]['matches']
tim = sample.iloc[0]['timeline']

In [39]:
rawdatas = mu.oracle_totalExecute('SELECT * FROM RAWDATA')

rawdatas.sort_values(['GAME_ID', 'PARTICIPANT_NUMBER'], ascending=[False, True], ignore_index=True, inplace=True)

matches = {a:[] for a in rawdatas.columns[6:]}
for idx, rec in enumerate(tqdm(rawdatas.iloc)):
    for col in [a for a in rawdatas.columns][6:]:
        matches[col].append(rawdataLoader(rec[col]))
for col in [a for a in rawdatas.columns][6:]:
    rawdatas[col] = matches[col]

oracle open!
oracle close!


620it [00:00, 1994.80it/s]


In [ ]:
ver = [[a, 1] for a in rawdatas['VERSION']]
ver_df = pd.DataFrame(ver, columns=['version', 'count'])
ver_count = ver_df.groupby('version').sum().sort_values('version', ascending=False)
ver_count

## 타임라인에 필요한 데이터:
* 챔피언 스킬 빌드
* 챔피언 시작 아이템
* 챔피언 아이템 빌드

### 챔피언 스킬 빌드 (채용!)

In [8]:
skill1 = mu.eventExtractor(sample.iloc[0], 'SKILL_LEVEL_UP', 1)

In [10]:
build1 = [a['skillSlot'] for a in skill1]
build1

[1, 3, 2, 1, 1, 4, 1, 3, 1, 3, 4, 3, 3, 2, 2]

### 챔피언 시작 아이템 (채용!)

In [50]:
def getStartItem(rawdata_series: pd.Series, participant_number: int):

    if participant_number not in range(1, 11): raise WrongParticipantNumber

    item_pur = mu.eventExtractor(rawdata_series, 'ITEM_PURCHASED', participant_number)
    purchased = [a['itemId'] for a in filter(lambda x: x['timestamp']<=120000, item_pur)]

    item_sold = mu.eventExtractor(rawdata_series, 'ITEM_SOLD', participant_number)
    sold = [a['itemId'] for a in filter(lambda x: x['timestamp']<=120000, item_sold)]

    item_undo = mu.eventExtractor(rawdata_series, 'ITEM_UNDO', participant_number)
    undo = [(a['beforeId'], a['afterId']) for a in filter(lambda x: x['timestamp']<=120000, item_undo)]

    for before, after in undo:
        if before == 0: purchased.remove(after)
        else: purchased.remove(before)
    for s in sold:
        purchased.remove(s)

    return purchased

In [59]:
# participant_number는 1~10
getStartItem(sample.iloc[0], 10)

[3858, 2003, 2003, 3340]

### 아이템 깊이 3 빌드 (채용!)

In [53]:
def itembuild(game: pd.Series, participant_number: int):
    if participant_number not in range(1, 11):
        raise WrongParticipantNumber
    return list(filter(lambda x: x in legend_items + mythic_itmes,
                        [a['itemId'] for a in mu.eventExtractor(game, 'ITEM_PURCHASED', participant_number)]))[:3]

In [58]:
# participant_number는 1~10
itembuild(sample.iloc[0], 1)

[6630, 3071, 3074]

### 아이템 깊이 무관 빌드 (미완성)

In [59]:
def partEvent(game: pd.Series, part: int, event: str, return_value: list) -> list:
    return [{k:v[k] for k in return_value} for v in list(filter(lambda x: x['participantId'] == part, mu.eventExtractor(game, event)))]

In [90]:
pid = 2
item_pur1 = partEvent(sample.iloc[0], pid, 'ITEM_PURCHASED', ['itemId','timestamp'])
item_dest1 = partEvent(sample.iloc[0], pid, 'ITEM_DESTROYED', ['itemId','timestamp'])
item_sold1 = partEvent(sample.iloc[0], pid, 'ITEM_SOLD', ['itemId','timestamp'])
item_undo1 = partEvent(sample.iloc[0], pid, 'ITEM_UNDO', ['beforeId', 'timestamp'])

In [105]:
pdf = pd.DataFrame(item_pur1)
pdf['type'] = 'p'
ddf = pd.DataFrame(item_dest1)
ddf['type'] = 'd'
sdf = pd.DataFrame(item_sold1)
sdf['type'] = 's'
udf = pd.DataFrame(item_undo1)
udf['type'] = 'u'

In [ ]:
item_pur1_list = [a['itemId'] for a in item_pur1]

for dest in [a['itemId'] for a in item_dest1]:
    item_pur1_list.remove(dest)
for sold in [a['itemId'] for a in item_sold1]:
    item_pur1_list.remove(sold)
for undo in [a['beforeId'] for a in item_undo1]:
    item_pur1_list.remove(undo)

item_pur1_list

In [73]:
p1 = [a['itemId'] for a in item_pur1]
p1.sort()
d1 = [a['itemId'] for a in item_dest1]
d1.sort()
s1 = [a['itemId'] for a in item_sold1]
s1.sort()
u1 = [a['beforeId'] for a in item_undo1]
u1.sort()

In [76]:
print(p1)
print(d1)
print(s1)
print(u1)

[1001, 1028, 1029, 1036, 1036, 1036, 1036, 1037, 1102, 2003, 2055, 2055, 2055, 2055, 2055, 2055, 2055, 2055, 2055, 3047, 3067, 3071, 3133, 3133, 6029, 6333, 6630]
[1001, 1029, 1036, 1036, 1036, 1036, 1037, 1102, 2003, 2055, 2055, 2055, 2055, 2055, 2055, 2055, 3067, 3133, 3133, 3340, 3513, 6029]
[]
[1028]


### 챔피언 밴률 (타임라인과 무관)

In [ ]:
ban_list = []
for loc in range(0, len(rawdatas), 10):
    matches = rawdataLoader(rawdatas.iloc[loc]['MATCHES'])
    bans = [a['championId'] for a in matches['game']['bans']]
    ban_list.append({'game_id':rawdatas.iloc[loc]['GAME_ID'], 'bans':bans})

In [ ]:
champs = mu.oracle_totalExecute('SELECT CHAMP_ID, CHAMP_NAME FROM CHAMPDATA')

oracle open!
oracle close!


In [ ]:
champs.sort_values('CHAMP_ID', ignore_index=True, inplace=True)
champs

In [ ]:
champs['BAN_TIME'] = champs.apply(lambda x: 0, axis=1)

In [ ]:
champ_id_list = [a for a in champs['CHAMP_ID']]

In [ ]:
ban_time = {a:0 for a in champ_id_list}
for lst in ban_list:
    for ban in lst['bans']:
        if ban == -1: continue
        ban_time[ban] += 1
ban_time

In [ ]:
ban_rate = {a:round((ban_time[a] / len(ban_list) * 100), 2) for a in ban_time}
ban_rate